In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import re
import requests
import pandas as pd
import numpy as np
import time
import os

In [2]:
all_ = pd.read_csv('../preprocessed_data/all_final_제거.csv', encoding='CP949')
회사명_2010 = all_[all_['회계년도']==2010]['회사명']

### 완성본

In [7]:
column_ = ['기업', '기사발행일', '기사제목', '뉴스기사본문']
뉴스_df = pd.DataFrame(columns=column_)

driver = webdriver.Chrome()
기업리스트 = 회사명_2010
기업리스트 = ['국순당']

start = '2010.01.01'
end = '2010.12.31'
start_= '20100101'
end_ = '20101231'

for 기업 in 기업리스트:
    # 나중에 뉴스_df와 concat할 임시 df 생성 및 임시 리스트들 생성
    column_ = ['기업', '기사발행일', '기사제목', '뉴스기사본문']
    임시_df = pd.DataFrame(columns=column_)    

    # 임시_df에 들어갈 리스트 생성
    본문리스트 = []
    날짜리스트 = []
    제목리스트 = []
    기업이름 = []

    # while 종료 조건으로 쓸 리스트 생성
    newslist = []
    datelist = []
    
    page = 1

    # 페이지수가 나와있지않으므로 맨끝에 page에 10씩더해서 계속 다음페이지로 이동
    while page < 150:

        url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query='+'"'+기업+'"'+'&sort=0&photo=0&field=0&pd=3&ds='+start+'&de='+end+'&cluster_rank=19&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from'+start_+'to'+end_+',a:all&start='+str(page)
        driver.get(url)

        response = requests.get(url)
        soup = BeautifulSoup(response.text, "lxml")

        # a태그중에서 class가 info인 것과 span태그에 class가 info 인 것 가져옴 
        news_titles = soup.select("a.info")
        dates = soup.select('span.info')

        # 네이버기사와 신문사기사 둘다 하이퍼링크가 있을경우는 기사당 news가 2개씩 네이버 기사가 없을 경우는 1개씩만 링크가 추출된다
        # 어림잡아 news_titles수가 10개 미만일 경우는 기사의 수가 5개에서 10개 미만이라는 뜻으로 해석하여 10개 이상만 링크와 기사날짜를 추출
        if len(news_titles) >= 10:

            for news in news_titles:
                title = news.attrs['href']
                newslist.append(title)

            for date in dates:
                news_date = date.text
                datelist.append(news_date)

            # 네이버 검색시 page number가 다음페이지로 갈때마다 1, 11, 21 이렇게 10씩 더해지는데 다음페이지가 없을경우 
            # 마지막 기사만을 포함한 같은 페이지를 계속 반환함. 따라서 기사와 날짜 둘 다 중복되서 저장될경우 종료함
            if (newslist[-1]==newslist[-2]) & (datelist[-1]==datelist[-2]):
                break

            page += 10

            time.sleep(0.5)
        
        # 기사수 5개 미만시 break, 기사없는기업 리스트에 저장함
        else:
            
            break
    
    # sid=101은 네이버 경제기사를 의미하는 듯함. 경제기사만 추출
    newslist = [news for news in newslist if 'sid=101' in news]
    
    # news_titles 뉴스기사 url 리스트가 존재시
    if news_titles:

        # 뉴스기사 url 자체에서는 text가 안가져와지는 특이사항 발생, 찾아보니 네이버기사는 인터넷에 떠야 페이지가 작동하는? 방식이라함
        # 셀레니움을 통해서 뉴스기사 url 주소로 창을 띄움
        for news in newslist:
            url = news
            driver.get(url)
            
            # 뉴스기사 url에서 본문과 제목, 기사작성날짜를 리스트에 저장함
            # 데이터프레임에 직접 행을 지정해주기는 번거로움..
            try:

                날짜 = driver.find_element_by_xpath('//*[@id="ct"]/div[1]/div[3]/div[1]/div/span').text
                날짜리스트.append(날짜)

                제목 = driver.find_element_by_xpath('//*[@id="ct"]/div[1]/div[2]/h2').text
                제목리스트.append(제목)

                본문 = driver.find_element_by_xpath('//*[@id="dic_area"]').text
                본문리스트.append(본문)
                time.sleep(1)

                기업이름.append(기업)
            
            ## 로딩이 안되서 데이터를 못가져올 경우를 대비해 sleep 3초 주고 다시 시도
            except:

                time.sleep(3)

                try:

                    날짜 = driver.find_element_by_xpath('//*[@id="ct"]/div[1]/div[3]/div[1]/div/span').text
                    날짜리스트.append(날짜)

                    제목 = driver.find_element_by_xpath('//*[@id="ct"]/div[1]/div[2]/h2').text
                    제목리스트.append(제목)

                    본문 = driver.find_element_by_xpath('//*[@id="dic_area"]').text
                    본문리스트.append(본문)
                    time.sleep(1)

                    기업이름.append(기업)
                
                # 그래도 데이터를 가져오지 못하는 경우는 페이지에 문제가 있다고 판단하여 PASS
                except:

                    pass

    # 혹시나 뉴스기사 url 리스트가 없을 경우는 pass                
    else:
        pass

    # 기업별로 가져온 날짜와 본문, 제목, 기업이름을 임시 데이터프레임으로 저장    
    임시_df.기사발행일 = 날짜리스트
    임시_df.뉴스기사본문 = 본문리스트
    임시_df.기사제목 = 제목리스트
    임시_df.기업 = 기업이름
    
    # 임시 데이터프레임을 뉴스 데이터프레임에 아래로 결합
    뉴스_df = pd.concat([뉴스_df, 임시_df])

# 앞에서 while 종료 조건이 같은 기사 2번저장인데 이럴 경우 중복으로 저장이 되야 종료되기때문에
# 중복기사 행 제거
뉴스_df.drop_duplicates(inplace=True)
뉴스_df.to_csv('뉴스_2010.csv', index=None)


C:\Users\dbswo\AppData\Local\Temp\ipykernel_5376\743684047.py:85: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  날짜 = driver.find_element_by_xpath('//*[@id="ct"]/div[1]/div[3]/div[1]/div/span').text
C:\Users\dbswo\AppData\Local\Temp\ipykernel_5376\743684047.py:88: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  제목 = driver.find_element_by_xpath('//*[@id="ct"]/div[1]/div[2]/h2').text
C:\Users\dbswo\AppData\Local\Temp\ipykernel_5376\743684047.py:91: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  본문 = driver.find_element_by_xpath('//*[@id="dic_area"]').text


In [6]:
뉴스_df

,기업,기사발행일,기사제목,뉴스기사본문
0,국순당,2010.12.28. 오후 7:35,[2010 하반기 히트상품] 국순당 / 우국생,국내산 쌀로 빚은 생막걸리로 인기\n\n[세계일보]\n백세주를 만드는 국순당이 지난...
1,국순당,2010.12.30. 오전 11:24,“배혜정 막걸리는 고난의 역사”,[머니투데이 문혜원 기자][[머니위크]Interview/ 배혜정 배혜정 누룩도가 대...
2,국순당,2010.12.20. 오전 9:57,국순당 '우리술상' 목동점 개장,(서울=연합뉴스) 이정내 기자 = 국순당(대표 배중호)은 목동역 인근에 우리술 전문...
3,국순당,2010.12.23. 오전 4:18,[2010 하반기 히트상품] 국순당 ‘우국생’,[서울신문]\n\n‘우국생’은 1년 이내 수확한 국내산 쌀로만 빚어 생막걸리 특유의...
4,국순당,2010.12.19. 오후 6:46,"국순당, 막걸리 매출이 백세주 추월",이제 '국순당=백세주'라는 공식은 깨야 할 듯하다. 올해 국순당 매출 중 절반 이상...
...,...,...,...,...
96,국순당,2010.09.27. 오전 7:52,"""아이디어 빌려요"" 국순당 주부모니터 선발",[아시아경제 이영규 기자]국순당(대표 배중호)이 제1기 전통주 주부 모니터 요원 1...
100,국순당,2010.10.11. 오전 8:52,"국순당, 실적부진 불구 성장성 여전-키움",[머니투데이 임상연 기자]키움증권은 11일 국순당에 대해 마케팅 비용 증가로 실적조...
101,국순당,2010.11.02. 오전 7:47,"국순당, 3Q 실적 기대치 크게 미달..`목표가↓`-신한","[이데일리 유용무 기자] 신한금융투자는 2일 국순당(043650)에 대해 ""3분기 ..."
118,국순당,2010.10.27. 오후 2:31,국순당 이산가족상봉행사 협찬,국순당이 제18차 남북 이산가족 상봉행사에서 '우국생' 막걸리와 백세주를 협찬주로 ...


,기업,기사발행일,기사제목,뉴스기사본문
217,가비아,2010.04.16. 오전 7:24,[16일 증권사 추천종목]호텔신라·현대중공업 등,◆우리투자증권\n\n<신규 추천종목>\n\n-호텔신라(국내경기 회복세와 원화 강세로...
218,가비아,2010.04.05. 오전 8:26,고성장 클라우드 컴퓨팅 수혜주는,(서울=연합뉴스) 구정모 기자 = 현대증권은 5일 정부 지원 강화와 글로벌 주요 기...
219,가비아,2010.04.04. 오후 12:12,"G밸리 기업들, 스마트폰 열풍에 ‘바쁘다 바빠’",G밸리(서울디지털산업단지)에서 스마트폰 애플리케이션 개발사업을 하고 있는 ‘포비커’...
220,가비아,2010.03.31. 오전 7:56,[31일 증권사 추천종목] 하나금융·KB금융 등,◆신한금융투자\n\n<신규 추천종목>\n\n-하나금융(순이자마진 상승 추세에 따른 ...
221,가비아,2010.08.09. 오전 11:30,[상한가 클럽] 회원 57% 실 계좌수익 공개 화제,증권방송 하이리치의 [상한가 클럽]이 성장형 실적주 발굴로 개인투자자들이 접근하기 ...
222,가비아,2010.08.09. 오전 11:30,[상한가 클럽] 회원 57% 실 계좌수익 공개 화제,증권방송 하이리치의 [상한가 클럽]이 성장형 실적주 발굴로 개인투자자들이 접근하기 ...
223,가비아,2010.08.06. 오후 2:30,중소형주 강한 반등 시그널! 공략법은?,최근 시장 흐름의 관심사는 지수가 상승하느냐 하락하느냐가 아니다. 이미 많은 전문가...
224,가비아,2010.02.22. 오전 7:26,"장 종료 후 ""이 공시 눈에 띄네""-코스닥(19일)",[아시아경제 박지성 기자]◆ 코미팜=BW 행사가액 5만8171원에서 5만1967원으...
225,가비아,2010.02.19. 오후 6:30,[코스닥 공시] MDS테크놀러지 外,"◇MDS테크놀러지=지난해 매출액 전년대비 4.6% 감소, 영업익 35% 증가 ◇엔빅..."
226,가비아,2010.02.19. 오후 5:36,[기업공시] 2월 19일,<유가증권시장>\n\n▦금호전기=토지 재평가 차액 770억원 발생 ▦현금배당=경동나...


### 오늘의 메모 / 증시일정 / 재테크 캘린더 / 특징주종합 등등 제거해야함

In [24]:
뉴스_df

,기업,기사발행일,기사제목,뉴스기사본문
0,세계투어,2010.01.07. 오전 9:34,"[특징주]세계투어↓, 클럽리치항공 합병 무산",[머니투데이 김지산 기자]세계투어가 클럽리치항공 흡수합병이 무산되자 7일 6% 이상...
1,세계투어,2010.01.06. 오후 5:33,"세계투어, 클럽리치항공 합병안 부결",[이데일리 김세형기자] 세계투어(047600)는 클럽리치항공과의 합병계약이 주주총회...
2,세계투어,2010.01.04. 오후 2:33,[특징주]자유투어 장중 상한가..여행주 급등,"[머니투데이 김지산 기자][여행수요 급증, 실적개선에 정부정책까지 가세]\n여행수요..."
3,세계투어,2010.01.11. 오후 9:00,이데일리 `내일의 경제일정`-대우건설 노조 기자회견,[이데일리 유용무기자] 내일(12일)은 대우건설 노조가 동국제강의 대우건설 전략적 ...
4,세계투어,2010.01.11. 오전 7:38,11일 증시일정,▶ 경제지표\n\n-중국 12월 생산자물가지수\n\n-중국 12월 소비자물가지수\n...
5,세계투어,2010.01.10. 오전 10:16,이번 주 재테크 캘린더(11~15일),[이데일리 윤도진기자]\n\n◇1월11일(월)\n\n▲추가상장\n- 다휘(05525...
6,세계투어,2010.01.10. 오전 10:20,[주간증시일정]11~15일,<11일>\n▷일반공모=서울식품(유진투자증권) 엔케이바이오\n▷추가상장=동양종금증권...
7,세계투어,2010.01.07. 오전 7:39,7일 증시일정,▶ 경제지표\n\n-미국 1월 신규 실업수당 청구건수\n\n-영국 BOE금리공시\n...
8,세계투어,2010.01.08. 오전 8:41,재테크 캘린더,◆ 1월 8일(금)\n＜부동산＞ ◇모델하우스 오픈＝청라지구 영화블렌하임 오피스텔(0...
9,세계투어,2010.01.06. 오전 7:49,6일 증시일정,▶ 경제지표\n\n-미국 12월 ISM비제조업지수\n\n▶ 추가·변경상장\n\n-씨...


### 임시

In [69]:
column_ = ['기업', '기사발행일', '뉴스기사본문']
뉴스_df = pd.DataFrame(columns=column_)

driver = webdriver.Chrome()
기업리스트 = ['세계투어']
행 = 0

start = '2010.01.01'
end = '2010.01.31'
start_= '20100101'
end_ = '20100131'

for 기업 in 기업리스트:

    column_ = ['기업', '기사발행일', '뉴스기사본문']
    임시_df = pd.DataFrame(columns=column_)    

    본문리스트 = []
    날짜리스트 = []
    기업이름 = []
    newslist = []
    datelist = []
    page = 1

    while True:
        
        url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query='+'"'+기업+'"'+'&sort=0&photo=0&field=0&pd=3&ds='+start+'&de='+end+'&cluster_rank=19&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from'+start_+'to'+end_+',a:all&start='+str(page)
        driver.get(url)

        response = requests.get(url)
        soup = BeautifulSoup(response.text, "lxml")

        news_titles = soup.select("a.info")
        dates_ = soup.select('span.info')

        a = 1
        for i in news_titles:
            title = i.attrs['href']
            if a % 2 ==0:
                newslist.append(title)
            a += 1
        
        for j in dates_:
            date = j.text
            datelist.append(date)

        if (newslist[-1]==newslist[-2]) & (datelist[-1]==datelist[-2]):
            break
        
        page += 10
    
    newslist = [news for news in newslist if 'sid=101' in news]

    for news in newslist:
        url = news
        driver.get(url)
        
        try:
            날짜 = driver.find_element_by_xpath('//*[@id="ct"]/div[1]/div[3]/div[1]/div/span').text
            날짜리스트.append(날짜)

            본문 = driver.find_element_by_xpath('//*[@id="dic_area"]').text
            본문리스트.append(본문)
            time.sleep(1)

            기업이름.append(기업)

            행 += 1

        except:
            pass
        
    임시_df.기사발행일 = 날짜리스트
    임시_df.뉴스기사본문 = 본문리스트
    임시_df.기업 = 기업이름
    
    뉴스_df = pd.concat([뉴스_df, 임시_df])
    

뉴스_df.to_csv('뉴스_df.csv', encoding='CP949', index=None)


C:\Users\dbswo\AppData\Local\Temp\ipykernel_12944\1166915341.py:59: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  날짜 = driver.find_element_by_xpath('//*[@id="ct"]/div[1]/div[3]/div[1]/div/span').text
C:\Users\dbswo\AppData\Local\Temp\ipykernel_12944\1166915341.py:62: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  본문 = driver.find_element_by_xpath('//*[@id="dic_area"]').text
